In [1]:
import os 
from dotenv import load_dotenv #importamos .env para introducir ahí nuestras credenciales
import pandas as pd
import requests 


In [10]:
ak_fm =	'45275fffebb8865ea08402cc6c12a3f9'
ak_secret = '400809380eba0959acd5f51ddcbccb0d'
url_fm = f'http://ws.audioscrobbler.com//2.0/?method=artist.getinfo&artist=Taylor+Swift&api_key={ak_fm}&format=json'  #la url llama a la información sobre artistas registrada en LAST.FM
p_fm = {'api_key':ak_fm,'shared_secret':ak_secret}
#PDT por alguna razón el .env da una respuesta 403 al llamar a la api

In [11]:
#llamamos a la api con una url que nos de todos los datos de Taylo Swift 
resp_fm = requests.get(url_fm)
resp_fm.status_code
print(f"Api nos ha dado esta respuesta: {resp_fm.status_code}")
print(f"Api nos ha dado este motivo: {resp_fm.reason}")


Api nos ha dado esta respuesta: 200
Api nos ha dado este motivo: OK


In [12]:
resp_fm.json() #vemos la información que contiene dentro y cumple las condiciones: artista, oyentes, reproducciones, biografía y similares

{'artist': {'name': 'Taylor Swift',
  'mbid': '20244d07-534f-4eff-b4d4-930878889970',
  'url': 'https://www.last.fm/music/Taylor+Swift',
  'image': [{'#text': 'https://lastfm.freetls.fastly.net/i/u/34s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'small'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/64s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'medium'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/174s/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'large'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'extralarge'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': 'mega'},
   {'#text': 'https://lastfm.freetls.fastly.net/i/u/300x300/2a96cbd8b46e442fc41c2b86b821562f.png',
    'size': ''}],
  'streamable': '0',
  'ontour': '1',
  'stats': {'listeners': '4857838', 'playcount': '2665271238'},
  'similar': {'artist': [{'name': 'S

In [14]:
df_pop_fm = pd.read_csv(r'/Users/mireia/Desktop/adalab/proyecto_spoty-api/Data/artistas_pop_2022_2023.csv') #cargamos el archivo cvs con los datos de los artistas pop, hasta aquí todo va a ser igual para todos los géneros

In [17]:
#creamos el diccionario donde almacenar toda la información de los artistas pop
fm_pop = {'artista':[],'oyentes':[],'reproducciones':[],'biografia':[],'similares':[]}
#creamos un contador que nos servirá para sacar solo los artistas similares que queremos en el bucle for desarrollado para ello
contador = 0
#desarrollamos el bucle for para interar por los datos del archivo csv, en este caso será un append simple por cada elemento del diccionario fm_pop menos para 'similares'
#df_pop_fm[0] hace referencia al nombre de la columna que contiene el nombre de los artistas en el archivo csv
for artista in df_pop_fm['0']:
    #llamamos a la api de nuevo, esta vez sustituyendo "Taylor+Swift" por {artista} para que itere por el archivo
    url_fm = f'http://ws.audioscrobbler.com//2.0/?method=artist.getinfo&artist={artista}&api_key={ak_fm}&format=json'
    respuesta_fm = requests.get(url_fm)
    #si la llamada es correcta, nos appendeará la información concreta en cada key del diccionario
    if respuesta_fm.status_code == 200:
        datos = respuesta_fm.json()
        fm_pop['artista'].append(datos['artist']['name'])
        fm_pop['oyentes'].append(datos['artist']['stats']['listeners'])
        fm_pop['reproducciones'].append(datos['artist']['stats']['playcount'])
        fm_pop['biografia'].append(datos['artist']['bio']['summary'])
        #para similares crearemos otro bucle for para poder limitar los artistas que nos devuelva a 3, ahora es cuando haremos uso del 'contador'
        for nombre in datos['artist']['similar']['artist']:
            if contador < 3:
                fm_pop['similares'].append(nombre['name']) 
                contador += 1        
    else:
        print(f'fallo {respuesta_fm.text}')
print(fm_pop)

{'artista': ['Noah Kahan', 'Djo', 'Sabrina Carpenter', 'SZA', 'd4vd', 'Josh Meloy', 'Harry Styles', 'Suki Waterhouse', 'Lil Durk', 'beabadoobee', 'Drake', 'Jid', 'Lizzy McAlpine', 'Taylor Swift', 'David Guetta', 'Wyatt Flores', 'Billie Eilish', 'Ernest', 'LECADE', 'omar apollo', 'Lainey Wilson', 'Imagine Dragons', 'Meghan Trainor', 'Post Malone', 'Benson Boone', 'Tate McRae', 'OneRepublic', 'The Weeknd', 'Cults', 'JVKE', 'Chance Peña', 'Becky G', 'Conan Gray', 'Feid', 'Rema', 'Hotel Ugly', 'Joji', 'Dhruv', 'Alok', 'Warren Zeiders', 'Riley Green', 'Jack Harlow', 'Sam Smith', 'NewJeans', 'Bryan Martin', 'Swedish House Mafia', 'Tory Lanez', 'Doja Cat', 'Beyoncé', 'Ozuna', 'Sean Paul', 'Yuridia', 'Bizarrap', 'Hailey Whitters', 'Rosalía', 'Ethel Cain', 'David Kushner', 'Treaty Oak Revival', 'Marshmello', 'Riton', 'Jin', 'Kylie Morgan', 'Matt Hansen', 'Shaboozey', 'Alesso', 'Connor Price', 'Kenshi Yonezu', 'sadie jean', 'MGK', 'Imanbek', 'Manuel Turizo', 'Tyler Hubbard', 'Alec Benjamin', 'Me

In [18]:
fm_pop.keys()

dict_keys(['artista', 'oyentes', 'reproducciones', 'biografia', 'similares'])

In [19]:
fm_pop

{'artista': ['Noah Kahan',
  'Djo',
  'Sabrina Carpenter',
  'SZA',
  'd4vd',
  'Josh Meloy',
  'Harry Styles',
  'Suki Waterhouse',
  'Lil Durk',
  'beabadoobee',
  'Drake',
  'Jid',
  'Lizzy McAlpine',
  'Taylor Swift',
  'David Guetta',
  'Wyatt Flores',
  'Billie Eilish',
  'Ernest',
  'LECADE',
  'omar apollo',
  'Lainey Wilson',
  'Imagine Dragons',
  'Meghan Trainor',
  'Post Malone',
  'Benson Boone',
  'Tate McRae',
  'OneRepublic',
  'The Weeknd',
  'Cults',
  'JVKE',
  'Chance Peña',
  'Becky G',
  'Conan Gray',
  'Feid',
  'Rema',
  'Hotel Ugly',
  'Joji',
  'Dhruv',
  'Alok',
  'Warren Zeiders',
  'Riley Green',
  'Jack Harlow',
  'Sam Smith',
  'NewJeans',
  'Bryan Martin',
  'Swedish House Mafia',
  'Tory Lanez',
  'Doja Cat',
  'Beyoncé',
  'Ozuna',
  'Sean Paul',
  'Yuridia',
  'Bizarrap',
  'Hailey Whitters',
  'Rosalía',
  'Ethel Cain',
  'David Kushner',
  'Treaty Oak Revival',
  'Marshmello',
  'Riton',
  'Jin',
  'Kylie Morgan',
  'Matt Hansen',
  'Shaboozey',
  '